Hammad Usmani

November 12th, 2025

_Please reference commit history for more details. No generative AI was utilized to create this work._

TODO:
- Mills primes table
  - A convinient table with the Mills prime can be provided
- Alternative verification method
  - The current method is sufficient
  - Alternatively, we can calculate the next (unverified) Mills constant and compare

In [1]:
!pip install gmpy2
!pip install mpmath

In [4]:
import mpmath #gmpy2 causes significant speedup, e.g. 35 mins -> 0.6 mins
import sys
import time

In [5]:
# Mills' constant A generates a sequence of primes via b(n)= floor(A^3^n). This sequence is a(n) = b(n+1)-b(n)^3.
# https://oeis.org/A108739
A108739 = [3, 30, 6, 80, 12, 450, 894, 3636, 70756, 97220, 66768, 300840, 1623568, 8436308]

In [6]:
def calc_prime(bn, c=3, a1=2):
    '''
    Calculates the prime number based on the sequence form described in OEIS A108739

    Parameters
    ----------
    bn list
        A list of integers according to the sequence form.
    c int
        From the form A^(c^n) this is the exponent.
        It's required that c >= 2.106 according to Caldwell and Cheng (see references).
    a1 int
        The first integer in the sequence according to Mill's constant.
    
    Returns
    int
        The calculated prime number based on Mills constant
    
    References
    ----------
    - https://cs.uwaterloo.ca/journals/JIS/VOL8/Caldwell/caldwell78.pdf
    '''
    stack = bn.copy()
    result = a1
    while (len(stack)>0):
        extension = stack.pop(0)
        result = result**c + extension
    return result

## Recreate "Mills - 555153 Digits.txt"

Before continuing, we formulate a method to recreate the accepted Mill's constant. Per the original README.md, **"Computed using the 13th Mills PRP. Verified using the 14th Mills PRP."** where the 13th Mills PRP relates to `b_13 = 300840` and `b_14 = 1623568` of the form `b_n`. Provided that the notation here is `A^(c^n)` where c = 3, we can attempt to recreate it.

To achieve this, we utilize the mpmath Python library to compute with arbitrary (unlimited) precision. Here are the steps based on the above notation for Mill's constant:

- `A^(3^n) = P_n`
  - _P_ is the prime number, related to _n_ and _b_n_.
  - We want _A_ (Mill's constant) to equal "Mills - 555153 Digits.txt"
- `pow(A^(3^n), 1/(3^n)) = pow(P_n, 1/(3^n))`
  - We take the _n_-th root on both sides to isolate _A_
  - This is the `mpmath.root()` function, see references
- `A = pow(P_n, 1/(3^n))`
  - Equivalent to `A = mpmath.root(P_n, (3^n))`

In order to verify the calculated Mills constant, we compare the next generated prime and analyze the error. In the A108739 sequence, we can calculate the next number in the sequence to get a verified prime. The error arises because the computed Mills constant doesn't have enough precision to calculate the `n+1` prime in the sequence. If we utilize the calculated Mills constant to generate the next prime number, it will be less than a certain number. This number must be exactly the next number in the A108739 sequence.

### References
- https://mpmath.readthedocs.io/en/1.3.0/index.html
- https://mpmath.readthedocs.io/en/1.3.0/functions/powers.html#root

In [47]:
# Increase Python's integer string conversion limit to arbitrarily high
# This will be used as a shortcut to get the number of digits for integers
sys.set_int_max_str_digits(20000000)

def calc_mills_constant(sequence, n, c=3):
    '''
    Reference the **Recreate "Mills - 555153 Digits.txt"** section.
    - The precision is calculated within this function.
    - Setting the precision may affect other calculations outside this function.
    - Minimality proof is based on the A108739 sequence.
    - Note that this function can take some time.

    Parameters
    ----------
    sequence list
        The sequence according to A108739
    n int
        The sequence number for the target
    c int
        The exponent value. Set to 3 by default
    
    Returns
    -------
    string
        The floating point value at the calculated precision.

    Notes
    -----
    When taking the floor, it converts the internal representation to the 
    decimal or output representation which can cause things to lose 
    precision. Therefore, we take the floor at an appropriate step
    in the operation.
    '''
    start = time.time()
    print('Calculating variables according to the sequence . . .')
    target = calc_prime(sequence[:(n - 1)])
    verification = calc_prime(sequence[:n])
    target_n = sequence[n - 2]
    verification_n = sequence[n - 1]
    precision = len(str(verification))
    verification_precision = 1000 # arbitrarily high number
    mpmath.mp.dps = (precision + verification_precision)
    print(f'Done. The precision is set at {mpmath.mp.dps} for verification.')
    print(f'Calculating Mills constant for the first {n} terms, this can take some time . . .')
    result = mpmath.nthroot(target, (c**n))
    end = time.time()
    print(f'Done! Total time elapsed: {int(end - start)/60:.2f} minutes.')
    print(f'Verifying . . .')
    # The following is when the floor of the function is applied prematurely
    # Causing the verification to fail
    # result_powered = mpmath.power(result, (c**(n+1)))
    # verification_value = mpmath.floor(result_powered, prec=0)
    verification_value = mpmath.power(result, (c**(n+1)))
    verification_diff = mpmath.floor(mpmath.fsub(verification, mpmath.fadd(verification_value, verification_n, exact=True), exact=True))
    verification_result = True if verification_diff == 0 else False
    if verification_result:
        final_result = str(result)[:-verification_precision]
        print(f'Done. Verification successful.')
        print(f'First 100 extra precision values are {str(result)[-verification_precision:][:100]}')
        return final_result
    else:
        print(f'Done. Verification failed. Difference is {verification_diff}')
        return result

In [48]:
# recreate Mills constant
A_13 = calc_mills_constant(A108739, 13)

Calculating variables according to the sequence . . .
Done. The precision is set at 556154 for verification.
Calculating Mills constant for the first 13 terms, this can take some time . . .
Done! Total time elapsed: 0.08 minutes.
Verifying . . .
Done. Verification successful.
First 100 extra precision values are 1786773721995395146671519757384924043663585766860054232059955380319096603375644421967278812057281379


In [49]:
with open('A_13.txt', 'w') as f:
    f.write(A_13)

## Upgrade Mills Constant
The A103879 sequence includes an additional extension we can use as verification to upgrade it.

In [50]:
# upgrade Mills constant
A_14 = calc_mills_constant(A108739, 14)

Calculating variables according to the sequence . . .
Done. The precision is set at 1666461 for verification.
Calculating Mills constant for the first 14 terms, this can take some time . . .
Done! Total time elapsed: 0.58 minutes.
Verifying . . .
Done. Verification successful.
First 100 extra precision values are 9462453719652241227829735119388611366538477692040200725532568260107511328667927015703490785983834716


In [51]:
with open('A_14.txt', 'w') as f:
    f.write(A_14)